In [1]:
import os
import pandas as pd
from settings import *

In [2]:
img_files = []
labels = []
label_code = []

for folder in ["sad", "happy", "relaxed"]:
    fname = os.path.join(IMAGE_DIR, folder)
    
    for im in os.listdir(fname):
        impath = os.path.join(fname, im)
        
        img_files.append(impath)
        labels.append(folder)
        label_code.append(FOLDERS.index(folder))

In [3]:
import pandas as pd

dataset = pd.DataFrame(dict(filename=img_files, label=labels, code=label_code))

In [4]:
dataset

,filename,label,code
0,images/sad/4618486329_c4dd489226_b.jpg,sad,1
1,images/sad/3921023490_d8e772150a_b.jpg,sad,1
2,images/sad/3429950849_2660a641d6_b.jpg,sad,1
3,images/sad/6957011199_2cf51d5464_b.jpg,sad,1
4,images/sad/8732538839_e9611eef88_b.jpg,sad,1
...,...,...,...
4692,images/relaxed/5515025332_4aeb4c7ed6_b.jpg,relaxed,2
4693,images/relaxed/34679620303_158a45a659_b.jpg,relaxed,2
4694,images/relaxed/3491188328_7162a9a7e4_b.jpg,relaxed,2
4695,images/relaxed/216405702_372588fa9d_b.jpg,relaxed,2


In [5]:
import torch

In [6]:
torch.manual_seed(0)

In [7]:
torch.cuda.is_available()

False

In [8]:
torch.backends.mps.is_available()

True

In [9]:
device = torch.device(DEVICE)

In [10]:
from torchvision.io import read_image
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotation, AutoAugment
from torch.utils.data import Dataset
import math

class DogDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.augments = [RandomHorizontalFlip(1), RandomRotation(90), RandomVerticalFlip(1)] + [AutoAugment()] * 4

    def __len__(self):
        return self.dataset.shape[0] * (len(self.augments) + 1)
    
    def classes(self):
        return self.dataset["code"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / self.dataset.shape[0])
        idx = idx % self.dataset.shape[0]
        
        row = self.dataset.iloc[idx,:]
            
        img_path = row["filename"]
        image = read_image(img_path)
        
        label = row["code"]
        
        if augment > 0:
            image = self.augments[augment - 1].forward(image)

        return image, int(label), img_path

In [11]:
data = DogDataset(dataset)

In [12]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(1))

In [13]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
EPOCHS = 50

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()
        
        self.bn = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=2),
            self.bn,
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 64, 1),
            self.bn,
            nn.Conv2d(64, 64, 3),
            self.bn,
            nn.Conv2d(64, 64, 1),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)
        )
        
        self.dense = nn.Sequential(
            nn.Linear(64 * 46 * 46, 64),
            nn.Linear(64, len(classes))
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1) 
        x = self.dense(x)
        return x

In [15]:
labels = data.classes()

In [16]:
model = NeuralNetwork(labels).to(device)

In [17]:
model

NeuralNetwork(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=1354

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [19]:
size = len(train.dataset)

for epoch in range(EPOCHS):
    for batch, (images, labels, img_paths) in enumerate(train): 
        optimizer.zero_grad()
        
        images = images.to(device)
        pred = model(images.float())
        
        labels = labels.to(device)
        loss = loss_fn(pred, labels)

        loss.backward()
        optimizer.step()

    loss, current = loss.item(), batch * len(images)
    print(f"loss: {loss:>7f}  [{epoch}]")

loss: 1.037006  [0]


KeyboardInterrupt: 

In [ ]:
#torch.save(model, 'dog_model.pth')

In [ ]:
#model = torch.load('dog_model.pth')

In [ ]:
all_preds = list()
all_labels = list()
all_paths = list()

with torch.no_grad():
    for batch, (images, labels, img_paths) in enumerate(test):
        
        images = images.to(device)
        outputs = model(images.float())
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.append(labels)
        all_preds.append(preds)
        all_paths.append(img_paths)

In [ ]:
import numpy as np

preds = np.concatenate([p.cpu().numpy() for p in all_preds])
labels = np.concatenate([p.cpu().numpy() for p in all_labels])
paths = np.concatenate([p for p in all_paths])

In [ ]:
((preds == labels).sum()) / len(labels)

In [ ]:
predictions = pd.DataFrame(dict(pred=preds, label=labels, path=paths))
predictions["correct"] = (predictions["pred"] == predictions["label"])

In [ ]:
predictions["emotion"] = predictions["pred"].apply(lambda x: FOLDERS[int(x)])

In [ ]:
predictions.groupby("emotion").apply(lambda x: x["correct"].sum() / x.shape[0])

In [ ]:
disp = predictions.iloc[:50,:].copy()

In [ ]:
def image_formatter(img):
    return f'<img src="{path}">'

In [ ]:
disp.style.format({'path': image_formatter})